<a href="https://colab.research.google.com/github/Luicazen/NLPTweets/blob/NN/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras import Input
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab') # Added this line
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv
Saving test.csv to test.csv


In [2]:
# Load the train.csv file into a DataFrame
df_train = pd.read_csv('train.csv')

# Display the first 5 rows of the DataFrame
display(df_train.head())

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
# Display basic information about the DataFrame
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [39]:
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

#Clean words
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize words
    words = word_tokenize(text)
    # Remove stop words, stem, and lemmatize
    cleaned_words = []
    for word in words:
        if word.isalpha() and word not in stop_words:
            # Apply stemming
            stemmed_word = stemmer.stem(word)
            # Apply lemmatization
            lemmatized_word = lemmatizer.lemmatize(stemmed_word)
            cleaned_words.append(lemmatized_word)
    return " ".join(cleaned_words)

# Apply the cleaning function to the 'text' column of df_train
df_train['cleaned_text'] = df_train['text'].apply(clean_text)

# Set nuWords to define the maximum number of words to keep (e.g., 10,000 common words)
numWords = 10000
# Set otherWords for out-of-vocabulary words
otherWords = "<unk>"
#Initialize a Tokenizer object
tokenizer = Tokenizer(num_words=numWords, oov_token=otherWords)

#Fit the tokenizer on the 'text' column of df_train
tokenizer.fit_on_texts(df_train['cleaned_text'])

#Convert the 'text' column into sequences of integers
cleanedSequences = tokenizer.texts_to_sequences(df_train['cleaned_text'])

#Determine a suitable maximum sequence length (maxlen)
maxLen = max([len(x) for x in cleanedSequences])

#Pad these sequences to ensure they all have the same length
cleaned_padded_sequences = pad_sequences(cleanedSequences, maxlen=maxLen, padding='post')

#Store the word index mapping
wordIndex = tokenizer.word_index

#update X
X = cleaned_padded_sequences
#Display original text
print(f"Original text sample: {df_train['text'].iloc[0]}")

#Display cleaned Text
print("Cleaned Text:", df_train['cleaned_text'].iloc[0])

#Display cleaned sequence
print(f"Tokenized sequence sample: {cleanedSequences[0]}")

#Display cleaned and padded sequence
print(f"Padded sequence sample: {cleaned_padded_sequences[0]}")

# Display vocab size
print(f"Vocabulary size: {len(wordIndex)}")


Original text sample: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Cleaned Text: deed reason earthquak may allah forgiv u
Tokenized sequence sample: [3526, 449, 223, 93, 1319, 2814, 10]
Padded sequence sample: [3526  449  223   93 1319 2814   10    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]
Vocabulary size: 12012


### Text Cleaning Function
Now, let's create a function that performs stop word removal, stemming, and lemmatization. This function will be applied to each text entry in our DataFrame.

### Re-Tokenize and Pad Cleaned Text
After cleaning, we need to re-apply the tokenizer and padding to prepare the data for the model. We'll use the same `tokenizer` and `maxLen` defined previously, but fit `tokenizer` on the `cleaned_text` column.

Now that we have cleaned and re-tokenized the text data, you will need to re-run the `train_test_split` cell, then re-define and re-train your model using `X` and `y`. This will allow the model to learn from the improved text representations.

In [40]:


# Define features (X) and target (y)
X = paddedSequences
y = df_train['target']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")

X_train shape: (6090, 33)
X_val shape: (1523, 33)
y_train shape: (6090,)
y_val shape: (1523,)


In [74]:
#Define the embedding dimension
embeddingDim = 128

#Create a Sequential model
model = Sequential([
    #Add an Input layer to explicitly define the input shape
    Input(shape=(maxLen,)),
    # 4. Add an Embedding layer
    # input_dim is the size of the vocabulary (num_words + 1 for 0-padding)
    # output_dim is the dimensionality of the dense embedding
    Embedding(input_dim=numWords + 1, output_dim=embeddingDim),

    # 5. Add a GlobalAveragePooling1D layer
    # This layer pools the sequence dimension, effectively taking the average of word embeddings
    GlobalAveragePooling1D(),

    # Add a Dense layer with 'relu' activation
    Dense(16, activation='relu'),

    # Add another Dense layer with 'relu' activation (added)
    Dense(8, activation='relu'),

    # 7. Add the final Dense layer with 1 neuron and 'sigmoid' activation for binary classification
    Dense(1, activation='sigmoid')
])

# Display the model summary to verify its architecture
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 23, 128)        │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_8      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 16)             │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,282,337 (4.89 MB)

 Trainable params: 1,282,337 (4.89 MB)

 Non-trainable params: 0 (0.00 B)

In [75]:
from tensorflow.keras.optimizers import Adam

epochs = 10
batch_size = 16

# Define a custom learning rate for the Adam optimizer
learning_rate = 0.001
custom_adam = Adam(learning_rate=learning_rate)

# 1. Compile the model using the custom Adam optimizer
model.compile(optimizer=custom_adam, loss='binary_crossentropy', metrics=['accuracy'])

# 2. Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    verbose=2
)

print("Model training complete.")

Epoch 1/10
381/381 - 7s - 18ms/step - accuracy: 0.5695 - loss: 0.6847 - val_accuracy: 0.5739 - val_loss: 0.6760
Epoch 2/10
381/381 - 6s - 17ms/step - accuracy: 0.5695 - loss: 0.6728 - val_accuracy: 0.5739 - val_loss: 0.6668
Epoch 3/10
381/381 - 5s - 14ms/step - accuracy: 0.5882 - loss: 0.6570 - val_accuracy: 0.6310 - val_loss: 0.6440
Epoch 4/10
381/381 - 6s - 17ms/step - accuracy: 0.6742 - loss: 0.6243 - val_accuracy: 0.6934 - val_loss: 0.6151
Epoch 5/10
381/381 - 5s - 14ms/step - accuracy: 0.7310 - loss: 0.5864 - val_accuracy: 0.7334 - val_loss: 0.5825
Epoch 6/10
381/381 - 7s - 17ms/step - accuracy: 0.7627 - loss: 0.5446 - val_accuracy: 0.7603 - val_loss: 0.5509
Epoch 7/10
381/381 - 5s - 14ms/step - accuracy: 0.7882 - loss: 0.5013 - val_accuracy: 0.7748 - val_loss: 0.5248
Epoch 8/10
381/381 - 6s - 15ms/step - accuracy: 0.8148 - loss: 0.4620 - val_accuracy: 0.7827 - val_loss: 0.5002
Epoch 9/10
381/381 - 7s - 18ms/step - accuracy: 0.8332 - loss: 0.4262 - val_accuracy: 0.7938 - val_loss:

## Evaluate Model

### Subtask:
Assess the performance of the trained neural network on the validation set using the `model.evaluate()` method to get the loss and accuracy.


In [76]:
from sklearn.metrics import f1_score
import numpy as np

# Make predictions on the validation set
y_pred_proba = model.predict(X_val)

# Convert probabilities to binary predictions using a threshold (e.g., 0.5)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate F1-score
f1 = f1_score(y_val, y_pred)

print(f"Model F1-score on validation set: {f1:.4f}")

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Model F1-score on validation set: 0.7343


#Run the Test file and create the submission.csv.

In [9]:

# Load the train.csv file into a DataFrame
df_test = pd.read_csv('test.csv')

# Display the first 5 rows of the DataFrame
display(df_test.head())

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# Display basic information about the DataFrame
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


### Preprocess Test Data
Apply the same cleaning function and tokenizer to the `df_test` DataFrame.

In [49]:
# Apply the cleaning function to the 'text' column of df_test
df_test['cleaned_text'] = df_test['text'].apply(clean_text)

# Convert the 'cleaned_text' column into sequences of integers using the *same tokenizer*
test_sequences = tokenizer.texts_to_sequences(df_test['cleaned_text'])

# Pad these sequences to ensure they all have the same length using the *same maxLen*
test_padded_sequences = pad_sequences(test_sequences, maxlen=maxLen, padding='post')

# Display original text
print(f"Original test text sample: {df_test['text'].iloc[0]}")

# Display cleaned Text
print("Cleaned test Text:", df_test['cleaned_text'].iloc[0])

# Display cleaned and padded sequence
print(f"Padded test sequence sample: {test_padded_sequences[0]}")

Original test text sample: Just happened a terrible car crash
Cleaned test Text: happen terribl car crash
Padded test sequence sample: [ 195 1611   55   21    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]


### Make Predictions on Test Data and Create Submission File

In [50]:
# Make predictions on the processed test data
y_test_pred_proba = model.predict(test_padded_sequences)

# Convert probabilities to binary predictions using a threshold (e.g., 0.5)
y_test_pred = (y_test_pred_proba > 0.5).astype(int)

# Create a submission DataFrame
submission_df = pd.DataFrame({'id': df_test['id'], 'target': y_test_pred.flatten()})

# Display the first few rows of the submission file
display(submission_df.head())

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully!")

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


,id,target
0,0,1
1,2,0
2,3,0
3,9,0
4,11,0


Submission file 'submission.csv' created successfully!
